# Introduction à python-lxml pour le traitement des sources textuelles

## lxml
[lxml](https://lxml.de/) est un portage de deux bibliothèques C, libxml2 et libxslt. C'est la librairie python la plus utilisée pour le traitement des sources XML (pour le HTML, il entre en concurrence avec d'autres outils comme BeautifulSoup par exemple). 

## Intérêts de Python-lxml
- Rapidité d'exécution
- Possibilité de 'brancher' le traitement à des outils puissants de TAL ou autres
- Un langage plus simple que XSLT
- Un traitement de corpus de documents *beaucoup* plus simple
  

## Limites de Python-lxml

### Version de XPath limitée
Python-lxml ne supporte que la version 1.0 de XPath: un certain nombre de fonctions ne sont donc pas disponibles; elles sont cependant souvent remplaçables par des fonctions *ad-hoc* relativement faciles à produire en python.

### Non-récursivité
Python-lxml ne fonctionne pas selon les principes de récursivité propre au fonctionnement en *templates* de XSLT, qui vous sera présenté par Ariane Pinche. Par conséquent, ce langage est beaucoup moins adapté aux tâches de transformation profonde des documents XML, pour de la production d'éditions au format XML ou pdf par exemple.

## Comparaison
Pour résumer, XSLT sera adapté aux tâche de transformations de données structurées complexes en d'autres données structurées d'un niveau de complexité similaire. Python sera plus adapté aux tâches d'extraction et d'analyse des données. Il est bien entendu possible de combiner et d'alterner ces deux langages dans une même chaîne de traitement.

| **Tâche**                                                                     | **LXML** | **XSLT**       |
|-------------------------------------------------------------------------------|-----------------|----------------|
| Extraction simple de données                                                  | **Faisable**        | **Faisable**       |
| Traitement texte/image                                                        | **Faisable**        | Difficile      |
| Traitement automatique du langage (TAL): enrichissement, extraction d'entités | **Faisable**        | Très difficile |
| Production d'une édition complexe aux formats du web                        | Très difficile | **Faisable** |
| Production d'une édition critique en LaTeX                                    | Très difficile     | **Faisable** |
| Collation automatisée                                                         | **Faisable**        | Difficile      |

## Les espaces de nommage
Les espaces de nommage ou espaces de noms sont un concept propre au XML. Le XML a une double caractéristique quant au contrôle d'un document. Un document XML est en effet **bien formé** quand il respecte les règles fondamentales du XML (pas de chevauchement des éléments, attributs séparés par des espaces, etc). 

Le contrôle des données d'un document ne peut se limiter à la vérification de la conformité du document aux règles XML. Il doit aussi être valide, selon les règles édictées par un **schéma** qui prendra plusieurs formes (RNC, RNG, DTD). Le XML étant un format industriel, il est fréquent que les acteurs qui l'utilisent soient de grosses institutions et consortiums qui produisent des **spécifications standards**: nous pouvons citer ALTO, PAGE, SVG, DublinCore, et bien sûr la TEI ou la MEI. 

L'espace de nommage permet de préciser la spécification du XML que l'on a choisie. Un espace de nommage est représenté par URI (Uniform Ressource Identifier) et par un préfixe (qui sera présenté plus bas).

### "Mélanger" les spécifications
Il n'est pas interdit (et il est parfois utile) de "mélanger" les différentes spécifications XML au sein d'un document. Ainsi par exemple, un document TEI pourra contenir dans son `teiHeader` un ensemble d'éléments propres à la spécification de DublinCore:
```XML
<?xml version="1.0" encoding="UTF-8"?>
<TEI xmlns="http://www.tei-c.org/ns/1.0" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
   xmlns:dc="http://purl.org/dc/elements/1.1/">
   <teiHeader>      
      <fileDesc>
         <titleStmt>
            <title>Titre du document</title>
            <author>Auteur du document</author>
         </titleStmt>
         <publicationStmt>
            <publisher>Éditeur du document</publisher>
            <date>2023-10-01</date>
         </publicationStmt>
         <sourceDesc>
            <p>Source description</p>
         </sourceDesc>
      </fileDesc>
      <xenoData>
         <rdf:RDF>
            <rdf:Description rdf:about="http://www.worldcat.org/oclc/606621663">
               <dc:title>The description of a new world, called the blazing-world</dc:title>
               <dc:creator>The Duchess of Newcastle</dc:creator>
               <dc:date>1667</dc:date>
               <dc:identifier>British Library, 8407.h.10</dc:identifier>
               <dc:subject>utopian fiction</dc:subject>
            </rdf:Description>
         </rdf:RDF>
      </xenoData>
   </teiHeader>
   <text>
      <body>
         <p>Some text here.</p>
      </body>
   </text>
</TEI>

```
***

```XML
<?xml version="1.0" encoding="UTF-8"?>
<TEI xmlns="http://www.tei-c.org/ns/1.0" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
   xmlns:dc="http://purl.org/dc/elements/1.1/">
   <teiHeader>      
      <fileDesc>
         <titleStmt>
            <title>Titre du document</title>
            <author>Auteur du document</author>
         </titleStmt>
         <publicationStmt>
            <publisher>Éditeur du document</publisher>
            <date>2023-10-01</date>
         </publicationStmt>
         <sourceDesc>
            <p>Source description</p>
         </sourceDesc>
      </fileDesc>
      <xenoData>
         <rdf:RDF>
            <rdf:Description rdf:about="http://www.worldcat.org/oclc/606621663">
               <dc:title>The description of a new world, called the blazing-world</dc:title>
               <dc:creator>The Duchess of Newcastle</dc:creator>
               <dc:date>1667</dc:date>
               <dc:identifier>British Library, 8407.h.10</dc:identifier>
               <dc:subject>utopian fiction</dc:subject>
            </rdf:Description>
         </rdf:RDF>
      </xenoData>
   </teiHeader>
   <text>
      <body>
         <p>Some text here.</p>
      </body>
   </text>
</TEI>

```

Dans l'exemple ci-dessus, repris à partir des *Guidelines* de la TEI, on combine la TEI, RDF et DublinCore. Nous observons en particulier que nous avons deux éléments `title` qui n'appartiennent pas au même espace de nommage. Le premier appartient à la TEI, et le second à DublinCore. La nécessité  de pouvoir correctement et explicitement distinguer ces éléments est claire. 

On note que les éléments DublinCore et RDF sont identifiés par un code `rdf` ou `dc` suivi des deux points `:`. Ce code s'appelle un préfixe. Il doit être déclaré dans le noeud racine du document xml à l'aide d'attribut `xmlns` (*xml namespace*): 

`<TEI xmlns="http://www.tei-c.org/ns/1.0" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">`

Le premier espace de nom ne contient pas de préfixe: il s'agit de l'espace de nommage par défaut. Tous les éléments qui ne seront pas identifiés par un préfixe se rattacheront à cet espace de nommage. 

### Indiquer ou pas le préfixe?
Vous aurez le choix dans le traitement des sources XML d'expliciter ou pas l'espace de noms des éléments que vous voulez transformer, notamment quand vous aurez à produire des requêtes XPath. Je vous recommande d'être explicite et de toujours indiquer le préfixe.

### Dans lxml
La gestion des espaces de noms se fait à l'aide d'un dictionnaire dans lxml: pour chaque espace de nommage, la clé est le préfixe, et la valeur l'URI de l'espace de nommage.

In [ ]:
tei_uri = "http://www.tei-c.org/ns/1.0"
dc_uri = "http://purl.org/dc/elements/1.1/"
namespaces_dict = {'tei': tei_uri,
                    'dc': dc_uri}

Nous n'utiliserons pas de sources contenant des éléments DublinCore, l'exemple est écrit ainsi pour vous donner une idée. Intéressons nous à la manipulation du document.

⚠ **ATTENTION** ⚠: Les premiers contacts avec les langages de transformation du XML peuvent être frustrants. Un adage de la communauté TEI est que si quelque chose ne marche pas comme prévu, il est fort probable que ça soit à cause de l'espace de nommage. Vérifiez-bien si tout est en règle de ce point de vue là en cas de problème!
***

# Manipulation de l'arbre XML

⚠ **IMPORTANT** ⚠: Les modifications de l'arbre XML se font en place (*in place*) et non pas par copie. Vous ne pourrez *pas* refaire l'exercice deux fois sans erreur si vous ne réinitialisez pas l'ensemble des variables.

## Parser le fichier
On va commencer par importer notre librairie.

In [ ]:
import lxml.etree as etree

La méthode `parse()` nous dispense d'ouvrir le fichier avec `open()`:

In [ ]:
fichier = 'fichier_xml_1.xml'
document_as_xml = etree.parse(fichier)
print(document_as_xml)

Nous obtenons un objet `lxml.etree._ElementTree`. Voyons les méthodes que l'on peut lui appliquer:

In [ ]:
print(dir(document_as_xml))

Nous nous intéresserons presque exclusivement à la méthode `xpath()` ici, mais notez les méthodes `xinclude` (parser les éléments inclus via un élément `xi:include` -- encore un autre espace de noms! -- ou xslt pour appliquer une feuille de transformation xslt à l'objet sélectionné.

Voyons maintenant comment fonctionnent les différents noeuds XML dans lxml. Je me sers ici de la documentation officielle de lxml ([ici](https://lxml.de/tutorial.html)).

### Récupération de la racine
La racine du document XML s'obtient avec la méthode `getroot()`:

In [ ]:
racine = document_as_xml.getroot()
print(racine)

### Les éléments XML sont des listes
Chaque noeud XML de notre arbre est un objet `lxml.etree.Element`, qui est une liste (dont les composants sont tous les éléments qu'il contient):

In [ ]:
print([item for item in racine])

On peut donc choisir un élément par son index dans la liste:

In [ ]:
header = racine[0]
print(header)

Le premier élément de notre arbre correspond à l'item d'index 0 de la liste: le `teiHeader`. On peut accéder à son non avec l'attribut `tag`:

In [ ]:
print(header.tag)

On voit à nouveau apparaître notre espace de nommage dans le nom de l'élément.

In [ ]:
print(racine[1])

Le second élément correspond au `body`. On note bien que l'espace de noms est systématiquement explicité par lxml. 

Enfin, on peut aller plus loin dans la structure en interrogeant les sous-listes:

In [ ]:
print(racine[0][0])

Ici, on va trouver le premier élément du `teiHeader`.

On peut donc effectuer toutes les opérations applicables aux listes: boucles, compréhensions, etc: c'est ce qui fait de lxml un outil extrêmement puissant. À l'inverse, le système d'imbrication n'est pas conservé, puisque le système de *templates* n'existe pas dans lxml.

### Les éléments portent leurs attributs comme des dictionnaires
On peut accéder aux attributs d'un élément avec l'attribut `attrib`. Voici un exemple (la première ligne de code est expliquée [plus bas](#Naviguer-dans-l'arbre:-la-méthode-xpath())).

In [ ]:
first_metsym = racine.xpath("//tei:metSym", namespaces=namespaces_dict)[0]
attributs = first_metsym.attrib
print(attributs)

On peut modifier la valeur d'un élément du dictionnaire comme on le fait normalement:

In [ ]:
attributs['value'] = 'valeur_corrigée'
attributs['nouvel_attribut'] = 'une_autre_valeur'
first_metsym.attrib

Nous verrons [plus bas](#Créer-un-attribut) une autre manière de créer un attribut.

## Sérialiser et enregistrer l'arbre
La sérialisation correspond à l'acte de sauvegarde d'une donnée à un format stable. Ici, plus simplement, il s'agit de l'enregistrement de notre document XML (qui a été désérialisé lors du *parsing*).

La méthode `etree.tostring()` permet de sérialiser un arbre importé: elle renvoie une chaîne de caractère en bytes qu'il va falloir convertir en chaîne. 

In [ ]:
xml_as_string = etree.tostring(racine, pretty_print=True).decode()
print(xml_as_string)

Nous pouvons maintenant enregistrer cette sérialisation:

In [ ]:
with open("fichier_xml_1.copie.xml", "w") as output_xml:
    output_xml.write(xml_as_string)

## Naviguer dans l'arbre: la méthode `xpath()`

La méthode `xpath(requête, namespaces)` qui est appliquée sur un noeud XML (la racine ou autre) est la plus adéquate ici. Elle accepte des **requêtes XPath 1.0**. Le résultat produit est à nouveau **une liste** (qui sera vide si l'élément cherché n'est pas présent dans le document) ou une valeur.

Attention aux espaces de noms avec le paramètre namespaces (au pluriel) qui doit renvoyer vers le dictionnaire recensant l'ensemble des espaces de noms du corpus. 

In [ ]:
all_lg = racine.xpath("descendant::tei:lg", namespaces=namespaces_dict)
print(all_lg)

⚠ **ATTENTION** ⚠: sélectionner un noeud sélectionne aussi la chaîne de caractère qui le suit directement (le *tail* en anglais) car elle est considérée comme rattachée à ce noeud. Cette caractéristique est propre à lxml et n'est pas partagée par les autres librairies de traitement de sources XML. Déplacer ou supprimer des noeuds peut donc mener à des conséquences imprévues et il faut l'avoir en tête à l'heure de travailler avec lxml.

In [ ]:
caesura = racine.xpath("descendant::tei:caesura", namespaces=namespaces_dict)[0]
print(etree.tostring(caesura, pretty_print=True).decode())

## Récupérer les noeuds textuels d'un élément XML
L'attribut `text` d'un objet `lxml.etree._Element` XML contient le texte inclus dans cet élément *jusqu'au premier élément XML qu'il contient*:

In [ ]:
foot_with_caesura = racine.xpath("descendant::tei:seg[tei:caesura]", namespaces=namespaces_dict)[0]
print(etree.tostring(foot_with_caesura).decode())

Voyons ce que contient l'attribut `text` ici:

In [ ]:
foot_with_caesura.text

Nous pouvons utiliser la méthode `itertext()` qui renvoie un générateur comprenant l'ensemble des noeuds textuels compris dans un noeud donné, mais cela revient souvent à écraser la structure et à la perdre:

In [ ]:
print([item for item in foot_with_caesura.itertext()])

**Nous touchons ici à une des limites de lxml**. Le traitement des sources structurées en XML-TEI avec une complexité importante (une source avec beaucoup d'abréviations, par exemple) est ardu et il est potentiellement plus simple de travailler avec XSLT pour ce genre de tâches, quitte à revenir à python plus tard.

## Créer un attribut
La création d'attributs set fait grâce à la méthode `set('attribut', 'valeur')` appliquée à l'élément choisi. 

Imaginons que nous voulions indiquer que l'identification de la césure dans le dernier vers à l'aide de l'élément `caesura` est assumée de science certaine. Nous allons utiliser l'attribut `@cert` pour cela:

In [ ]:
caesuras = racine.xpath("descendant::tei:caesura", namespaces=namespaces_dict)
print(caesuras)

La méthode xpath renvoie (presque) toujours une liste même si elle ne contient qu'un élément: il faut donc veiller à sélectionner cet élément explicitement en indiquant sa position dans la liste.

In [ ]:
caesuras[1].set('cert', 'high')

Regardons le résultat en ciblant le couplet:

In [ ]:
modified_lg = racine.xpath("descendant::tei:lg[@type='couplet']", namespaces=namespaces_dict)[0]
print(etree.tostring(modified_lg, pretty_print=True).decode())

## Créer un élément et l'insérer dans l'arbre
Une fois encore, le point le plus important n'est pas la création de nouveaux éléments, mais l'attribution de l'espace de nommage correct à ces éléments. Dans le cas inverse, vous ne pourrez pas les manipuler comme des noeuds TEI, par exemple.

La manipulation des espaces de nommage pour la production de nouveaux éléments est un peu distincte que pour le requêtage des arbres XML. On va ici créer un nouveau dictionnaire sans indiquer le préfixe afin qu'il ne soit pas ajouté dans le nom de l'élément lors de la sérialisation (voir [ici](https://lxml.de/tutorial.html#namespaces)).

In [ ]:
tei_ns = "{%s}" % tei_uri
NSMAP_no_prefix = {None: tei_uri}

Créons une note que nous allons ajouter au titre de notre sonnet.

In [ ]:
ma_note = etree.Element(tei_ns + "note", nsmap=NSMAP_no_prefix)

On peut lui ajouter le texte que l'on veut, via l'attribut `text`:

In [ ]:
ma_note.text = "Ceci est ma première note concernant le sonnet."

In [ ]:
print(etree.tostring(ma_note, pretty_print=True).decode())

On peut maintenant l'insérer à l'endroit que l'on souhaite. Pour ce faire, on va utiliser la méthode `insert(index, sous-element)` appliquée à l'élément parent.

In [ ]:
element_parent = racine.xpath("descendant::tei:body/descendant::tei:head", namespaces=namespaces_dict)[0]
element_parent.insert(1, ma_note)

Regardons ce que cela a donné:

In [ ]:
print(etree.tostring(element_parent, pretty_print=True).decode())

Il existe d'autres méthodes pour créer et/ou insérer des noeuds: `SubElement()`, `append()`, `addnext()`, `addprevious()`. La documentation vous permettra de choisir la méthode la plus adaptée à vos besoins.

## Supprimer un élément
Il n'y a pas de manière directe de supprimer un élément. Il existe une méthode indirecte avec la fonction `strip_elements()`. Elle suppose d'identifier l'élément à supprimer, de sélectionner son parent, et de supprimer l'élément ciblé.  Il est important d'ajouter le paramètre `with_tail=False` pour éviter de supprimer les noeuds textuels qui suivent directement l'élément à supprimer (https://stackoverflow.com/a/41359368). Supprimons notre premier élément `caesura`:

In [ ]:
etree.strip_elements(caesuras[0].getparent(), '{http://www.tei-c.org/ns/1.0}caesura', with_tail=False)

In [ ]:
modified_lg = racine.xpath("descendant::tei:lg[@type='quatrain'][3]", namespaces=namespaces_dict)[0]
print(etree.tostring(modified_lg, pretty_print=True).decode())

In [ ]:
## Document final
xml_as_string = etree.tostring(racine, pretty_print=True).decode()
with open("fichier_xml_1_final.xml", "w") as output_xml:
    output_xml.write(xml_as_string)